從 https://github.com/makeyourownmaker/QuasicrystalGifs/tree/main 抄襲過來的 QuasiCrystal 產生器。修改成我自己喜歡的形式，原始邏輯和原本的程式相同。但是有些參數會亂數產生，使得程式每次執行生成的圖片都不一樣。


In [ ]:
import numpy as np
from math import pi
from math import sqrt
from math import ceil
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from google.colab import drive
from matplotlib.animation import PillowWriter
import random
import matplotlib
import json
import matplotlib.image as mpimg

def RandBool():
  if random.randint(0,1) == 0:
    return True
  else:
    return False

def GetStdLineArray(gridX,gridY,waves,freq,offset=0,custom_waves=None):
  allwaves=(np.arange(0,pi,pi/waves)+offset/waves) if custom_waves is None else custom_waves
  xcos=gridX*np.cos(allwaves)[:,np.newaxis,np.newaxis]
  ysin=gridY*np.sin(allwaves)[:,np.newaxis,np.newaxis]
  ret=(xcos+ysin)*freq;
  return ret

def GetPolarLineArray(gridX,gridY,waves,freq,offset=0,custom_waves=None,sq=True,log=True):
  allwaves=(np.arange(0,pi,pi/waves)+offset/waves) if custom_waves is None else custom_waves
  theta=np.arctan2(gridX, gridY)
  r=gridX*gridX+gridY*gridY
  if sq: r=np.sqrt(r)
  if log: r=np.log(r)
  r[np.isinf(r) is True] = 0
  tcos=theta*np.cos(allwaves)[:,np.newaxis,np.newaxis]
  rsin=r*np.sin(allwaves)[:, np.newaxis, np.newaxis]
  ret=(tcos+rsin)*freq;
  return ret

def GeneratePolarFunc(sq,log):
  def retFunc(gridX,gridY,waves,freq,offset=0,custom_waves=None):
    return GetPolarLineArray(gridX,gridY,waves,freq,offset,custom_waves,sq,log)
  return retFunc

def GenerateQuasiCrystalPhase(waves,freq,width,height,scale=1,offset=0,jump_zero=False,custom_waves=None,func=GetStdLineArray):
  r=pi*scale
  if jump_zero:
    dx = np.arange(-r+r/width, r+r/width, 2*r/width,dtype=np.float64)
    dy = np.arange(-r+r/height, r+r/height, 2*r/height,dtype=np.float64)
  else:
    dx = np.arange(-r, r, 2*r/width,dtype=np.float64)
    dy = np.arange(-r, r, 2*r/height,dtype=np.float64)
  xv, yv = np.meshgrid(dx, dy)
  return func(xv,yv,waves,freq,offset,custom_waves)

#  bdmode=['hsv','overlay','soft']
#  cmap=['Accent', 'Blues', 'BrBG', 'BuGn',    'BuPu', 'CMRmap', 'Dark2', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'Paired',
#       'Pastel1', 'Pastel2', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds',
#       'Set1', 'Set2', 'Set3', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg',
#       'bwr', 'cividis', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar',
#       'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'inferno', 'jet', 'magma', 'nipy_spectral',
#       'ocean', 'pink', 'plasma', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'tab10', 'tab20', 'tab20b', 'tab20c', 'terrain',
#       'turbo', 'twilight', 'twilight_shifted', 'viridis', 'winter']
# or add _r, like 'gray_r'

def GenerateImageProducer(blend_mode,azdeg,altdeg,vexag):
  AllBlendMode=['hsv','overlay','soft']
  func=None
  if blend_mode not in AllBlendMode:
    def func(image,cmap):
      return image
  else:
    ls = LightSource(azdeg=azdeg, altdeg=altdeg)
    def func(image,cmap):
      return ls.shade(image,cmap=plt.get_cmap(cmap),blend_mode=blend_mode,vert_exag=vexag)
  return func

def GenerateQuasiCrystalImage(data,image,cmap,imageFunc,calcFunc=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  if imageFunc is None: imageFunc = GenerateImageProducer(None,0,0,0)
  image[:] = np.sum(calcFunc(data), axis=0)
  rgb = imageFunc(image,cmap)
  im = plt.imshow(rgb,cmap=cmap)
  return im

def ShowQuasiCrystalImage(data,image,cmap,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None):
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)
  return GenerateQuasiCrystalImage(data,image,cmap,imageFunc)

def ShowQuasiCrystalAnimate(data,image,figure,frames,delay,cmap,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None,path=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  phases = np.arange(0,2*pi,2*pi/frames)
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)

  def animate_func(i):
    adata=data+phases[i]
    im = GenerateQuasiCrystalImage(adata,image,cmap,imageFunc,calcFunc)
    return [im]

  ani = animation.FuncAnimation(figure,animate_func,frames=frames,interval=delay)
  if path is not None:
    ani.save(path,writer='pillow')
  return ani;

def CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,figure,frames,delay,cmap,func,jump_zero=False,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None,path=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)

  ls=len(scale_array)
  lw=len(waves_array)
  lf=len(freq_array)
  lo=len(offset_array)
  lp=len(phase_array)

  def animate_func(i):
    data=GenerateQuasiCrystalPhase(waves_array[i%lw],freq_array[i%lf],width,height,scale_array[i%ls],offset_array[i%lo],jump_zero,custom_waves=None,func=func)
    adata=data+phase_array[i%lp]
    im = GenerateQuasiCrystalImage(adata,image,cmap,imageFunc,calcFunc)
    return [im]

  ani = animation.FuncAnimation(figure,animate_func,frames=frames,interval=delay)
  if path is not None:
    ani.save(path,writer='pillow')
  return ani

class ParamSaver:
  def __init__(self):
    self.params={}

  def output(self):
    return json.dumps(self.params,indent=4)

  def load(self, json_string):
    self.params=json.loads(json_string)

  def set(self, key, value):
    if value is not None:
      self.params[key]=value

  def get(self, key):
    try:
      return self.params[key]
    except:
      return None

  def getBool(self, key):
    v=self.get(key)
    if v is None:
      return RandBool()
    else:
      return v

  def getInt(self, key, min, max=None):
    v=self.get(key)
    if v is None:
      if max==None:
        return min
      else:
        return random.randint(min,max)
    else:
      return int(v)

  def getString(self,key,default):
    v=self.get(key)
    if v is None:
      return default
    else:
      return v

  def getFloat(self, key, min, max=None):
    v=self.get(key)
    if v is None:
      if max==None:
        return min
      else:
        return random.random() * (max - min) + min
    else:
      return float(v)

  def getChoice(self, key, choices, toChoice=True):
    v=self.get(key)
    if v is None and toChoice:
      return random.choice(choices)
    else:
      return v

  def getNumericArray(self, key, min, maxStep, length, periodic=False):
    v=self.get(key)
    if v is list:
      return v;
    if maxStep==0 or length <= 0 or RandBool():
      return [min]
    if type(maxStep)==int:
      step=random.randint(1,maxStep) if maxStep > 0 else random.randint(-maxStep,0)
    else:
      step=maxStep
    if periodic==False:
      return [min+i*step for i in range(0,length)]
    if length%2==0:
      half_ret = [min+i*step for i in range(0,int(length/2)+1)]
      return half_ret+half_ret[::-1][1:-1]
    else:
      helf_ret = [min+i*step for i in range(0,int(length/2)+1)]
      return helf_ret+helf_ret[::-1][:-1]

  def hasNumericArray(self, key):
    v=self.get(key)
    return v is list

  def generateInfoString(self):
    w=self.getInt("waves",0,)
    f=self.getInt("freq",0,)
    m=self.getString("calcFunc","None")
    cmap=self.getString("colormap","None")
    if m=="polar" or m=="logpolar":
      s=self.getInt("scale",0)
      return "waves_%d_freq_%d_%s%d_colormap_%s" % (w,f,m,s,cmap)
    else:
      return "waves_%d_freq_%d_colormap_%s" % (w,f,cmap)

  def save(self, path):
    v=self.output()
    try:
      with open(path, 'w', encoding='utf-8') as f:
        f.write(v)
      print(f"成功將字串寫入到 {path}")
    except IOError as e:
      print(f"寫入檔案時發生錯誤: {e}")

  def loadFile(self, path):
    try:
      with open(path, 'r', encoding='utf-8') as f:
        self.load(f.read())
      print(f"成功從 {path} 讀取字串")
    except IOError as e:
      print(f"讀取檔案時發生錯誤: {e}")

In [ ]:
json_string='{}'
input=ParamSaver()
input.load(json_string)
output=ParamSaver()

In [ ]:
width = input.getInt("width",512)
height = input.getInt("height",512)
scale=input.getInt("scale",1,input.getInt("maxScale",10))
waves = input.getInt("waves",4,input.getInt("maxWaves",128))
freq = input.getInt("freq",2,input.getInt("maxFreq",64))
offset = input.getFloat("offset",0,pi if RandBool() else None)
jump_zero = input.getBool("jump_zero")
AllFunc=['default','polar','sqrtpolar','logpolar','sqrtlogpolar']
calcFuncName=input.getChoice("calcFunc",AllFunc)

if calcFuncName=='polar':
  func = GeneratePolarFunc(False,False)
elif calcFuncName=='logpolar':
  func = GeneratePolarFunc(False,True)
elif calcFuncName=='sqrtpolar':
  func = GeneratePolarFunc(True,False)
elif calcFuncName=='sqrtlogpolar':
  func = GeneratePolarFunc(True,True)
else:
  func = GetStdLineArray

output.set("width",width)
output.set("height",height)
output.set("scale",scale)
output.set("waves",waves)
output.set("freq",freq)
output.set("offset",offset)
output.set("jump_zero",jump_zero)
output.set("calcFunc",calcFuncName)

inner=GenerateQuasiCrystalPhase(waves,freq,width,height,scale,offset,jump_zero,func=func)

image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ShowQuasiCrystalImage(inner,image,'gray')

In [ ]:
print(output.output())

In [ ]:
cmap = input.getChoice("colormap",plt.colormaps())
AllBlendModes = ['hsv','overlay','soft']
blend_mode=input.getChoice("blend_mode",AllBlendModes,RandBool())

if blend_mode in AllBlendModes:
  azdeg=input.getInt("azdeg",0,360)
  altdeg=input.getInt("altdeg",0,90)
  vexag=input.getInt("vexag",1,10)
else:
  azdeg=None
  altdeg=None
  vexag=None

output.set("colormap",cmap)
if blend_mode in AllBlendModes:
  output.set("blend_mode",blend_mode)
  if azdeg is not None:
    output.set("azdeg",azdeg)
  if altdeg is not None:
    output.set("altdeg",altdeg)
  if vexag is not None:
    output.set("vexag",vexag)
else:
  output.set("blend_mode","None")

image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ShowQuasiCrystalImage(inner,image,cmap,blend_mode,azdeg,altdeg,vexag)

In [ ]:
print(output.output())

In [ ]:
frames=input.getInt("frames",24)
delay=input.getInt("delay",100)
matplotlib.rcParams['animation.embed_limit']=50
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

ani=ShowQuasiCrystalAnimate(inner,image,fig,frames,delay,cmap,blend_mode,azdeg,altdeg,vexag)
HTML(ani.to_jshtml())

In [ ]:
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ani=ShowQuasiCrystalAnimate(inner,image,fig,frames,delay,cmap,blend_mode,azdeg,altdeg,vexag)
name=output.generateInfoString()+".gif"

ani.save(name, writer='pillow')

In [ ]:
frames=input.getInt("frames",20,30)
delay=input.getInt("delay",100)
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

offset_array=input.getNumericArray("offset_array",offset,offset,frames,RandBool())
phase_array=input.getNumericArray("phase_array",0,2*pi/frames,frames)
waves_array=input.getNumericArray("waves_array",waves,input.getInt("max_waves_step",4),frames,RandBool())
freq_array=input.getNumericArray("freq_array",freq,input.getInt("max_freq_step",6),frames,RandBool())
scale_array=input.getNumericArray("scale_array",scale,input.getInt("max_scale_step",ceil(scale*0.1)),frames,RandBool())

testTrival=not(input.hasNumericArray("waves_array") and input.hasNumericArray("freq_array") and input.hasNumericArray("scale_array"))
while testTrival:
  if len(waves_array)>1 or len(freq_array)>1 or len(scale_array)>1:
    break
  waves_array=input.getNumericArray("waves_array",waves,input.getInt("max_waves_step",4),frames,RandBool())
  freq_array=input.getNumericArray("freq_array",freq,input.getInt("max_freq_step",6),frames,RandBool())
  scale_array=input.getNumericArray("scale_array",scale,input.getInt("max_scale_step",ceil(scale*0.1)),frames,RandBool())


output.set("frames",frames)
output.set("delay",delay)
output.set("waves_array",waves_array)
output.set("freq_array",freq_array)
output.set("offset_array",offset_array)
output.set("phase_array",phase_array)
output.set("scale_array",scale_array)


ani=CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,fig,frames,100,cmap,func,jump_zero,blend_mode,azdeg,altdeg,vexag)
HTML(ani.to_jshtml())

In [ ]:
output.save(output.generateInfoString()+'.json')
print(output.output())

In [ ]:
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ani=CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,fig,frames,100,cmap,func,jump_zero,blend_mode,azdeg,altdeg,vexag)
name=output.generateInfoString()+"_v.gif"

ani.save(name, writer='pillow')
